In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/MyDrive/colab_projects/nlp/imdb/nbs/

/content/gdrive/MyDrive/colab_projects/nlp/imdb/nbs


In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [4]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [5]:
df = pd.read_csv("../data/imdb.csv")

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df.shape

(50000, 2)

In [8]:
df_train = df[:25000]
df_test = df[25000:]

In [9]:
len(df_train), len(df_test)

(25000, 25000)

In [10]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100

In [11]:
labels_index = {'positive':1, 'negative':0}

In [12]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_train.review.values)
train_sequences = tokenizer.texts_to_sequences(df_train.review.values)
test_sequences = tokenizer.texts_to_sequences(df_test.review.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 90624 unique tokens.


In [13]:
X_train = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
y_train = to_categorical(np.asarray(df_train.sentiment.map(labels_index)))
y_test = to_categorical(np.asarray(df_test.sentiment.map(labels_index)))

In [14]:
len(word_index), MAX_NUM_WORDS

(90624, 20000)

In [15]:
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1

In [16]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model, Sequential

In [17]:
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(labels_index), activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=4)

score, acc = model.evaluate(X_test, y_test)
print('Test accuracy with CNN:', acc)

Epoch 1/4
196/196 [==============================] - 26s 43ms/step - loss: 0.6059 - acc: 0.6227
Epoch 2/4
196/196 [==============================] - 8s 42ms/step - loss: 0.2547 - acc: 0.8992
Epoch 3/4
196/196 [==============================] - 8s 42ms/step - loss: 0.1757 - acc: 0.9346
Epoch 4/4
782/782 [==============================] - 4s 6ms/step - loss: 0.3451 - acc: 0.8738
Test accuracy with CNN: 0.8737999796867371


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 100)         2000100   
_________________________________________________________________
conv1d (Conv1D)              (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 128)           82048     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0

In [20]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

model.fit(X_train, y_train,
          batch_size=128,
          epochs=4)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Training the RNN
Epoch 1/4
196/196 [==============================] - 22s 104ms/step - loss: 0.6251 - accuracy: 0.6385
Epoch 2/4
196/196 [==============================] - 21s 106ms/step - loss: 0.2874 - accuracy: 0.8847
Epoch 3/4
196/196 [==============================] - 21s 107ms/step - loss: 0.1847 - accuracy: 0.9326
Epoch 4/4
782/782 [==============================] - 19s 24ms/step - loss: 0.4362 - accuracy: 0.8208
Test accuracy with RNN: 0.8208400011062622


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 2,691,842
Trainable params: 2,691,842
Non-trainable params: 0
_________________________________________________________________
